# Pyntacle

This Jupyter notebook provides an example of using the Python packages [gravis](https://pypi.org/project/gravis) and [Pyntacle](http://pyntacle.css-mendel.it). The .ipynb file can be found [here](https://github.com/robert-haas/gravis/tree/master/examples).

## References

- [Pyntacle website](http://pyntacle.css-mendel.it)
  - [Tutorials](http://pyntacle.css-mendel.it/#tutorials) (Case studies)
  - [Documentation](http://pyntacle.css-mendel.it/#docs)

Note: Pyntacle builds on top of **igraph**. It uses igraph Graph objects and the usual igraph methods are available for them.

## Installation

- With [pip](https://pypi.org/project/pyntacle/): `pip install pyntacle`
- With [conda](https://anaconda.org/search?q=pyntacle): `conda install -c conda-forge -c bfxcss pyntacle`

## Import

In [ ]:
import os
import warnings
warnings.simplefilter('ignore')  # ignore various CUDA warnings

In [ ]:
import igraph  # for graph generation
import pyntacle
import pyntacle.algorithms.local_topology
import pyntacle.algorithms.global_topology
import pyntacle.io_stream.generator
import pyntacle.io_stream.importer

import gravis as gv

## Quick start

### Example 1

In [ ]:
def assign_properties(g):
    # Pyntacle builds on top of igraph. The following calculations are provided by igraph!

    # Centrality calculation
    node_centralities = g.betweenness()
    edge_centralities = g.edge_betweenness()
    
    # Community detection
    communities = g.community_fastgreedy().as_clustering().membership

    # Node properties: Size by centrality, color by community
    colors = ['red', 'blue', 'green', 'orange', 'pink', 'brown', 'yellow', 'cyan', 'magenta', 'violet']
    g.vs['size'] = [5.0 + val / 10.0 for val in node_centralities]
    g.vs['color'] = [colors[community_index % len(colors)] for community_index in communities]
    
    # Edge properties: Size by centrality, color by community (within=community color, between=black)
    g.es['size'] = [0.5 + val / 50.0 for val in edge_centralities]
    g.es['color'] = [colors[communities[i] % len(colors)] if communities[i] == communities[j] else 'black'
                     for i, j in g.get_edgelist()]
    

# Create a graph
filepath = os.path.join('data', 'pyntacle_graph.adjm')
g = pyntacle.io_stream.importer.PyntacleImporter.AdjacencyMatrix(
    file=filepath, header = True, sep = "\t")

# Assign properties
assign_properties(g)

# Plot it
gv.d3(g, zoom_factor=0.2)

## Graph construction

- API reference: [pyntacle.io_stream](http://pyntacle.css-mendel.it/html/pyntacle.io_stream.html#)

### 1) Manual graph construction

In [ ]:
# ~ Not supported by Pyntacle, possible via igraph ~

### 2) Algorithmic graph construction

- API Reference: [pyntacle.io_stream.generator](http://pyntacle.css-mendel.it/html/pyntacle.io_stream.generator.html#module-pyntacle.io_stream.generator)

In [ ]:
generator = pyntacle.io_stream.generator.PyntacleGenerator
num_nodes = 50


# Erdos-Renyi model - arg1: number of nodes, arg2: number of edges (or rewiring probability)
g = generator.Random([num_nodes, 80])

# Barabasi-Albert model - arg1: number of nodes, arg2: average number of edges per node
g = generator.ScaleFree([num_nodes, 2])

# Watts-Strogatz model - arg1: dimension of the lattice, arg2: size of the lattice among all dimensions
#                        arg3: distance k between nodes, arg4: node rewiring probability p
g = generator.SmallWorld([2, 10, 1, 0.1])

# Tree topology as in Wolfram alpha docs - arg1: number of nodes, arg2: number of children per node
g = generator.Tree([num_nodes, 3])


gv.d3(g)

### 3) Graph loading from an internal collection

In [ ]:
# ~ Not supported by Pyntacle, possible via igraph ~

### 4) Graph import and export

- API reference: [pyntacle.io_stream](http://pyntacle.css-mendel.it/html/pyntacle.io_stream.html)

#### Import

- API reference: [pyntacle.io_stream.importer](http://pyntacle.css-mendel.it/html/pyntacle.io_stream.importer.html#module-pyntacle.io_stream.importer)

In [ ]:
importer = pyntacle.io_stream.importer.PyntacleImporter

filepath = os.path.join('data', 'pyntacle_graph.adjm')
g = importer.AdjacencyMatrix(file=filepath, header=True, sep='\t')

#### Export

- API reference: [pyntacle.io_stream.exporter](http://pyntacle.css-mendel.it/html/pyntacle.io_stream.exporter.html#module-pyntacle.io_stream.exporter)

In [ ]:
# TODO

## Basic graph inspection

### 1) Graph and its properties

In [ ]:
print(type(g))  # pyntacle uses igraph as basis

### 2) Nodes and their properties

In [ ]:
# TODO

### 3) Edges and their properties

In [ ]:
# TODO

## Calculating graph measures and metrics

### 1) Quantitative measures

- API reference: [pyntacle.algorithms](http://pyntacle.css-mendel.it/html/pyntacle.algorithms.html)

#### 1.a) Graph properties

- API reference: [Global topology](http://pyntacle.css-mendel.it/html/pyntacle.algorithms.global_topology.html#module-pyntacle.algorithms.global_topology)

In [ ]:
gt = pyntacle.algorithms.global_topology.GlobalTopology

value = gt.average_closeness(g)
value = gt.average_clustering_coefficient(g)
value = gt.average_degree(g)
value = gt.average_eccentricity(g)
value = gt.average_radiality(g)
value = gt.average_radiality_reach(g)
value = gt.components(g)
value = gt.density(g)
value = gt.diameter(g)
value = gt.pi(g)
value = gt.radius(g)
value = gt.weighted_clustering_coefficient(g)        

#### 1.b) Node properties

- API reference: [Local topology](http://pyntacle.css-mendel.it/html/pyntacle.algorithms.local_topology.html#module-pyntacle.algorithms.local_topology)

In [ ]:
lt = pyntacle.algorithms.local_topology.LocalTopology

values = lt.betweenness(g)
values = lt.closeness(g)
values = lt.clustering_coefficient(g)
values = lt.degree(g)
values = lt.eccentricity(g)
# values = lt.eigenvector_centrality(g)  # fails
values = lt.pagerank(g)
values = lt.radiality(g)
values = lt.radiality_reach(g)

#### 1.c) Group properties

- API reference: [Local topology](http://pyntacle.css-mendel.it/html/pyntacle.algorithms.local_topology.html#module-pyntacle.algorithms.local_topology)

In [ ]:
node_names = g.vs['name']
used_node_names = node_names[:5]

value = lt.group_betweenness(g, used_node_names)
value = lt.group_closeness(g, used_node_names)
value = lt.group_degree(g, used_node_names)

### 2) Structure inference

In [ ]:
# TODO

## Graph visualization

In [ ]:
# TODO